In [1]:
import torch
import wandb
from pathlib import Path
import sys
sys.path.append("../src")

from torchinfo import summary
from torch import optim

import os
from dataset import ShakespearDataset
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10
from torchvision.transforms import v2

from quickai.trainer import Trainer
from quickai.utils import model_size, load_from_checkpoint
from quickai.callbacks import OverfitCallback, EarlyStoppingCallback
from quickai.logger import WandbLogger
from quickai.dataset import MapDataset

from module import BigramLanguageModule
import torch.nn as nn
from torch.nn import functional as F
import settings as s

In [2]:
data_path = Path("../data")
logs_path = Path("../logs")
logs_path.mkdir(exist_ok=True)

In [3]:
logger = WandbLogger(
    project_name=s.project_name,
    config={
        "model": s.model,
        "dataset": s.dataset,
        "max_epochs": s.max_epochs,
        "optimizer": s.optimizer,
        "lr_scheduler": s.lr_scheduler,
        "test_run": s.test_run,
        "transfer_learning": s.transfer_learning
    },
    logs_path=logs_path,
    offline=s.wandb_offline
)

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [4]:
cpu_count = os.cpu_count()
# cpu_count = 7

dataset = ShakespearDataset(data_path/"shakespear.txt", s.dataset["context_size"])

train_dataset, val_dataset = random_split(
    dataset, [s.dataset["train_split"], s.dataset["val_split"]]
)

train_dataloader = DataLoader(
    train_dataset, batch_size=s.dataset["batch_size"], shuffle=True, num_workers=cpu_count)
val_dataloader = DataLoader(
    val_dataset, batch_size=s.dataset["batch_size"],  num_workers=cpu_count)

In [5]:
for step, batch in enumerate(train_dataloader):
    x, y = batch
    break

x.shape, y.shape

AttributeError: Can't pickle local object 'ShakespearDataset.setup.<locals>.encode'

In [5]:
module = BigramLanguageModule(dataset.num_chars)

optimizer = optim.AdamW(
    params=module.model.parameters(),
    weight_decay=s.optimizer["weight_decay"]
)

try:
    if s.lr_scheduler["name"] == "OneCycleLR":
        lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(
            optimizer=optimizer,
            max_lr=s.lr_scheduler["max_lr"],
            epochs=s.max_epochs,
            steps_per_epoch=len(train_dataloader),
        )

        print(s.lr_scheduler["name"])
except TypeError:
    lr_scheduler = None
    print("lr_scheduler is None!")

lr_scheduler is None!


In [6]:
trainer = Trainer(
    module=module,
    logger=logger,
    optimizer=optimizer,
    callbacks=[],
    logs_path=logs_path,
    fast_dev_run=s.fast_dev_run,
    limit_batches=s.limit_batches,
    lr_scheduler=lr_scheduler,
    save_checkpoint_type="best_val",
    num_workers=cpu_count
)

Using device: cpu!


In [ ]:
try:
    trainer.fit(train_dataloader, val_dataloader)
except KeyboardInterrupt as e:
    print("Run stopped!")
finally:
    wandb.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Sanity checking with fast dev run!


model_architecture,BigramLanguageModel(...


In [19]:
decode(model.generate(idx=torch.ones((1, 1), dtype=torch.long), max_new_tokens=100).tolist()[0])

" '?KKRkuHgRZOm iQInL$!VeL\nB?KN$cqa\nMNVTA:Mi!iPRHFoFJ3Lso&'CgOmGGtWlJNAvWOXnqaDeKID:msUQiACHl'FNQAwUWA"